In [1]:
from bs4 import BeautifulSoup
import asyncio
from aiohttp import ClientSession
import numpy as np
import pandas as pd
from functools import partial
from aiohttp import ClientConnectorError
import json
import re
from bs4 import NavigableString
from pprint import pprint

## Scrape Web Content

```
.headline-article-main
.key-facts p
.content-body-text p
.tags a
time
```

In [106]:
async def get_html_data(trail, session):
    """Access CNN webpage"""
    url = trail
    print(url)
    attempts = 0
    while attempts < 3:
        try:
            res = await session.get(url)
            try:
                html = await res.text()
            except UnicodeDecodeError:
                return trail, None
            soup = BeautifulSoup(html, 'html.parser')
            headline = soup.select(".headline-article-main")[0].text
            facts = [fact.text for fact in soup.select(".key-facts p")]
            body = "\n".join([sent.text for sent in soup.select(".content-body-text p")]).replace("\n\n", "\n")
            tag = [tag.text for tag in soup.select(".tags a")]
            try:
                time = re.search(r"\(.*\d\d\d\d", soup.select("time")[0].text).group(0)
            except:
                time = None
            attrs = {
                "headline": headline,
                "keyfacts": facts, 
                "content": body,
                "tags": tag,
                "time": time
                    }
            return trail, attrs
        except ClientConnectorError:
            attempts += 1
            print("Connector error occurred!")
    if attempts == 3:
        return trail, None

In [107]:
async def gather_results(curr, step, trails):
    """Launch scrape tasks and collect results"""
    tasks = []
    async with ClientSession() as session:
        for trail in trails[curr: curr + step]:
            task = asyncio.ensure_future(get_html_data(trail, session))
            tasks.append(task)

        responses = await asyncio.gather(*tasks)
        # you now have all response bodies in this variable
        return responses


def process_df(future, curr, step):
    """Save scrape results in json files"""
    cache = {k: v for k, v in future.result()}
#     cache = future.result()
    if len(cache) == 0:
        raise RuntimeError("Empty response!")
    else:
#         json.dump(cache, open("./data/scraped/scraped_{0}_{1}.json".format(curr, curr + step), "w"))
        global temp
        temp = cache
        print("got it!")


# df = pd.read_csv("./data/ratings.csv")
# app_ids = df.loc[:, "app_id"].astype(str)

start = 0
end = 1
step = 1

trails = ['http://cnnespanol.cnn.com/2017/09/07/florida-se-prepara-para-evacuaciones-masivas-y-atascos-por-el-huracan-irma/']

for curr in range(start, end, step):
    print("loading data from {0} to {1}".format(curr, curr + step))
    loop = asyncio.get_event_loop()
    future = asyncio.ensure_future(gather_results(curr, step, trails))
    future.add_done_callback(partial(process_df, curr=curr, step=step))
    loop.run_until_complete(future)

loading data from 0 to 1
http://cnnespanol.cnn.com/2017/09/07/florida-se-prepara-para-evacuaciones-masivas-y-atascos-por-el-huracan-irma/
got it!


In [108]:
print(temp[list(temp.keys())[0]]['content'])

(CNN) - El gobernador de la Florida, Rick Scott, dice que se espera que haya más evacuaciones masivas en todo el estado antes del huracán Irma, la poderosa tormenta de categoría 5 que arado a través del Caribe.
"Si se le dice que evacue, salga rápidamente", dijo Scott el miércoles. "Podemos esperar evacuaciones adicionales mientras esta tormenta continúa acercándose a nuestro estado".
LEE: Minuto a minuto: el huracán Irma deja 9 muerto en su devastador paso por el Caribe
Basado en el camino proyectado por Irma, que incluye la costa este de Florida, la enorme tormenta podría crear una de las mayores evacuaciones masivas de la historia de Estados Unidos, dijo el meteorólogo de la CNN, Dave Hennen. Miami-Dade, Broward y Palm Beach son hogar de cerca de 6 millones de personas.
El condado de Monroe, hogar de los Cayos de la Florida, ya ha ordenado evacuaciones obligatorias. El condado de Broward, que incluye a Fort Lauderdale, emitió una evacuación obligatoria el miércoles para las áreas al

In [109]:
temp[list(temp.keys())[0]]

{'content': '(CNN) - El gobernador de la Florida, Rick Scott, dice que se espera que haya más evacuaciones masivas en todo el estado antes del huracán Irma, la poderosa tormenta de categoría 5 que arado a través del Caribe.\n"Si se le dice que evacue, salga rápidamente", dijo Scott el miércoles. "Podemos esperar evacuaciones adicionales mientras esta tormenta continúa acercándose a nuestro estado".\nLEE: Minuto a minuto: el huracán Irma deja 9 muerto en su devastador paso por el Caribe\nBasado en el camino proyectado por Irma, que incluye la costa este de Florida, la enorme tormenta podría crear una de las mayores evacuaciones masivas de la historia de Estados Unidos, dijo el meteorólogo de la CNN, Dave Hennen. Miami-Dade, Broward y Palm Beach son hogar de cerca de 6 millones de personas.\nEl condado de Monroe, hogar de los Cayos de la Florida, ya ha ordenado evacuaciones obligatorias. El condado de Broward, que incluye a Fort Lauderdale, emitió una evacuación obligatoria el miércoles 

## Scrape Web Url

```
http://http://cnnespanol.cnn.com/2017/09/05/  -- variable date
.entry-title a
```

In [118]:
async def get_html_data(trail, session):
    """Access CNN daily archives webpage"""
    url = trail
    print(url)
    attempts = 0
    while attempts < 3:
        try:
            res = await session.get(url)
            try:
                html = await res.text()
            except UnicodeDecodeError:
                return trail, None
            soup = BeautifulSoup(html, 'html.parser')
            urls = [a['href'] for a in soup.select(".entry-title a") if not re.search(r'/video/', a['href'])]
            return trail, urls
        except ClientConnectorError:
            attempts += 1
            print("Connector error occurred!")
    if attempts == 3:
        return trail, None

async def gather_results(curr, step, trails):
    """Launch scrape tasks and collect results"""
    tasks = []
    async with ClientSession() as session:
        for trail in trails[curr: curr + step]:
            task = asyncio.ensure_future(get_html_data(trail, session))
            tasks.append(task)

        responses = await asyncio.gather(*tasks)
        # you now have all response bodies in this variable
        return responses


def process_df(future, curr, step):
    """Save scrape results in json files"""
    cache = {k: v for k, v in future.result()}
#     cache = future.result()
    if len(cache) == 0:
        raise RuntimeError("Empty response!")
    else:
#         json.dump(cache, open("./data/scraped/scraped_{0}_{1}.json".format(curr, curr + step), "w"))
        global temp
        temp = cache
        print("got it!")


# df = pd.read_csv("./data/ratings.csv")
# app_ids = df.loc[:, "app_id"].astype(str)

start = 0
end = 3
step = 3

trails = ['http://cnnespanol.cnn.com/2017/09/05/',
         'http://cnnespanol.cnn.com/2017/09/06/',
         'http://cnnespanol.cnn.com/2017/09/07/']

for curr in range(start, end, step):
    print("loading data from {0} to {1}".format(curr, curr + step))
    loop = asyncio.get_event_loop()
    future = asyncio.ensure_future(gather_results(curr, step, trails))
    future.add_done_callback(partial(process_df, curr=curr, step=step))
    loop.run_until_complete(future)

loading data from 0 to 3
http://cnnespanol.cnn.com/2017/09/05/
http://cnnespanol.cnn.com/2017/09/06/
http://cnnespanol.cnn.com/2017/09/07/
got it!


In [120]:
pprint(temp)

{'http://cnnespanol.cnn.com/2017/09/05/': ['http://cnnespanol.cnn.com/2017/09/05/asperger-nino-argentina-madres-whatsapp-grupo-polemica-indignacion/'],
 'http://cnnespanol.cnn.com/2017/09/06/': ['http://cnnespanol.cnn.com/2017/09/06/corea-del-norte-estaria-movilizando-un-misil-intercontinental-cree-seul/',
                                           'http://cnnespanol.cnn.com/gallery/fotos-las-mejores-imagenes-de-la-visita-del-papa-francisco-a-colombia/',
                                           'http://cnnespanol.cnn.com/2017/09/06/estrella-del-futbol-se-queda-sin-club-por-un-retraso-en-su-transferencia-de-14-segundos/'],
 'http://cnnespanol.cnn.com/2017/09/07/': ['http://cnnespanol.cnn.com/2017/09/07/facebook-vendio-publicidad-a-cuentas-falsas-rusas-durante-campana-presidencial-de-ee-uu-de-2016/',
                                           'http://cnnespanol.cnn.com/gallery/fotos-las-devastadoras-imagenes-de-los-estragos-que-causo-irma-en-el-caribe/',
                               